# GANs Image Generation

In this notebook, we will explore how Generative Adversarial Networks (GANs) generate images. We will use a pretrained GAN model (BigGAN) to generate images from random noise.

## Instructions
1. Run the code below to generate an image from random noise.
2. Modify the latent vector to generate different images.
3. Experiment with generating different images by altering the latent vector and visualizing the results.


In [1]:
import torch
from transformers import BigGAN
from torchvision import transforms
from PIL import Image

# Load pretrained BigGAN model
model = BigGAN.from_pretrained('biggan-deep-256')

# Generate random latent vector (noise)
latent_vector = torch.randn(1, 128)  # 128 is the size of the input vector for BigGAN

# Generate image
with torch.no_grad():
    generated_image = model(latent_vector)

# Convert the tensor to a displayable image
generated_image = generated_image.squeeze().cpu().numpy()
generated_image = (generated_image * 255).astype('uint8')
Image.fromarray(generated_image)

ImportError: cannot import name 'BigGAN' from 'transformers' (/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/transformers/__init__.py)

In [2]:
# Experiment with different latent vectors
latent_vector = torch.randn(1, 128)  # Change this to explore different random vectors
with torch.no_grad():
    generated_image = model(latent_vector)
generated_image = generated_image.squeeze().cpu().numpy()
generated_image = (generated_image * 255).astype('uint8')
Image.fromarray(generated_image)

## Reflection

Now that you have generated images, write a brief report reflecting on your observations:

1. How did the generated images change when you modified the latent vector?
2. What patterns did you notice in the generated images? Were they realistic?
3. How does the process of generating images from noise differ from traditional image generation methods?
4. What challenges or limitations did you observe with the GAN model?